In [2]:
import torch
from torchvision import models

In [3]:
#The Pascal cased model names are the classes
#The smaller case names of the corresponding classes are the convenience function that returns the instance of the model class
# For example, densenet121, densenet161, densenet169, densenet201, all are instances of DenseNet class but with different number of layers – 121,161,169 and 201, respectively  
dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'quantization',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'utils',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vg

# Download the pre-trained model

In [4]:
#Load the model
alexnet = models.alexnet(pretrained=True)

In [5]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

# Create a Transform to be applied on the inputs to the Model

In [6]:
from torchvision import transforms

In [7]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406] , std=[0.229,0.224,0.225])
])

# Load and pre-process an Image

In [8]:
from PIL import Image

In [29]:
input_img = Image.open("..//data//images//lion.jpg")


In [30]:
input_img_transform = transform(input_img)
print(" Input shape : ", input_img_transform.shape)
batch_transform = torch.unsqueeze(input_img_transform, 0) #Unsqueeze the image to send it out as a batch for predictions
print(" batch shape : ", batch_transform.shape)

 Input shape :  torch.Size([3, 224, 224])
 batch shape :  torch.Size([1, 3, 224, 224])


# Run Model Inference

In [31]:
# Set the model in eval mode
alexnet.eval()
output_vector = alexnet(batch_transform)
output_vector.shape

torch.Size([1, 1000])

In [32]:
#Output _vector is a vector of 1000 classes
#We need the class labels

with open('..//data//docs//imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]
labels = [k.split(', ')[1] for k in classes]

In [33]:
#The output is a row vector so we find the max in dim : 1 (column) 
_, index = torch.max(output_vector,1) 
print(labels[index])
print(index.shape)

# Obtain the probabilities using the softmax function along dim : 1 i.e column %
percent = torch.nn.functional.softmax(output_vector,dim=1)[0] * 100
print(percent[index])


lion
torch.Size([1])
tensor([99.9972], grad_fn=<IndexBackward>)


In [34]:
# Sort the indices in decending order of probabilities
_, indices = torch.sort(output_vector,1,True)
print(indices.shape)

# Print the top 6 probabilities
[(labels[idx], percent[idx].item()) for idx in indices[0,:6]]

torch.Size([1, 1000])


[('lion', 99.99720001220703),
 ('Persian_cat', 0.0006670920993201435),
 ('Pekinese', 0.0005496992380358279),
 ('Shetland_sheepdog', 0.0005062207928858697),
 ('ram', 0.0004189471947029233),
 ('ox', 8.515155059285462e-05)]

# Code-snippets for Testing

In [33]:
a  =  torch.tensor([[1,2,3],
                    [4,5,6],
                    [7,8,9]])
b = torch.unsqueeze(a,1)
print(a.shape)
print(b)
print(b.shape)

torch.Size([3, 3])
tensor([[[1, 2, 3]],

        [[4, 5, 6]],

        [[7, 8, 9]]])
torch.Size([3, 1, 3])


In [71]:
c = torch.tensor([[1,2,3,4,5,6]])
print(c)
_,d = torch.max(c,0)
d

tensor([[1, 2, 3, 4, 5, 6]])


tensor([0, 0, 0, 0, 0, 0])